In [21]:
import numpy as np
import scipy.sparse as sp

import sys
sys.path.append('/home/koshirshov/koshirshov/RecBot')
import config
from pandas_utils import load_data
import pandas as pd

In [4]:
path = config.proceed_data_path
users_df = load_data(path + 'users_processed.pkl',)
items_df = load_data(path + 'items_processed.pkl',)
train_interactions = load_data(path + 'train_interactions.pkl',)
test_interactions = load_data(path + 'test_interactions.pkl',)
sample_submission = load_data(path + 'sample_submission_processed.pkl',)

In [22]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]

# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


In [26]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating', 
                  'studios', 'countries']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,release_year_cat_1990-2000,release_year_cat_2000-2010,release_year_cat_2010-2012,release_year_cat_2012-2014,...,"countries_франция, япония",countries_хорватия,countries_чехия,countries_чили,countries_швейцария,countries_швеция,countries_эстония,countries_юар,countries_югославия,countries_япония
0,10711,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
def prepare_csr_matrix(train_interactions, quantity_column = 'quantity'):
    train_interactions = train_interactions[train_interactions['watched_pct'] > 25]
    train_interactions = train_interactions[train_interactions['user_id'].isin(set(users_ohe_df['user_id']))]
    train_interactions = train_interactions[train_interactions['item_id'].isin(set(items_ohe_df['item_id']))]
    train_interactions[quantity_column] = 1
    all_users = set(train_interactions['user_id'])
    
    good_users = train_interactions.groupby('user_id')[quantity_column].sum()
    good_users = set(good_users[good_users >= 5].index)
    good_items = train_interactions.groupby(['item_id'])[quantity_column].sum()
    good_items = set(good_items[good_items >= 10].index)

    train_interactions = train_interactions[train_interactions['user_id'].isin(good_users)]
    train_interactions = train_interactions[train_interactions['item_id'].isin(good_items)]
    users_not_in_matrix = all_users - set(train_interactions['user_id'])

    good_users = set(train_interactions['user_id'])
    good_items = set(train_interactions['item_id'])
    users_mapping = dict(zip(good_users, range(len(good_users))))
    items_mapping = dict(zip(good_items, range(len(good_users))))
    users_mapping_reverse = {value:key for (key,value) in users_mapping.items()}
    items_mapping_reverse = {value:key for (key,value) in items_mapping.items()}

    train_interactions.loc[:, "le_user_id"] = train_interactions.loc[:, 'user_id'].map(users_mapping)
    train_interactions.loc[:, "le_item_id"] = train_interactions.loc[:, 'item_id'].map(items_mapping)
    return train_interactions, users_mapping, items_mapping, users_mapping_reverse, items_mapping_reverse, users_not_in_matrix

train_interactions, users_mapping, items_mapping, users_mapping_reverse, items_mapping_reverse, users_not_in_matrix = prepare_csr_matrix(train_interactions)

In [59]:
users_with_interactions_in_train = (set(train_interactions['user_id'])&set(test_interactions['user_id'])) - users_not_in_matrix
users_without_interactions_in_train = (set(test_interactions['user_id']) - users_with_interactions_in_train).union(users_not_in_matrix)
users_with_interactions_in_train = [user for user in users_with_interactions_in_train]
users_without_interactions_in_train = [user for user in users_without_interactions_in_train]

In [60]:
items_ohe_df = items_ohe_df[items_ohe_df['item_id'].isin(items_mapping.keys())]
users_ohe_df = users_ohe_df[users_ohe_df['user_id'].isin(users_mapping.keys())]

In [61]:
interactions_vec = np.zeros((train_interactions.le_user_id.nunique(), 
                             train_interactions.le_item_id.nunique()))

for user_id, item_id in zip(train_interactions.le_user_id, train_interactions.le_item_id):
    interactions_vec[user_id, item_id] += 1
    
res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [64]:
print(train_interactions.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(train_interactions.user_id.nunique())
print(users_ohe_df.user_id.nunique())

set(items_ohe_df.item_id.unique()) - set(train_interactions.item_id.unique())

5293
5293
126005
126005


set()